## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有8種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

```
觀察狀態 = 'dry', 'dry', 'dry'
Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104

Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888
Sunny, Sunny, Rainy: 0.4*(0.6)*0.6*(0.6)*0.4*(0.1) = 0.003456
Sunny, Rainy, Sunny: 0.4*(0.6)*0.4*(0.1)*0.3*(0.6) = 0.001728

Sunny, Rainy, Rainy: 0.4*(0.6)*0.4*(0.1)*0.7*(0.1) = 0.000672
Rainy, Rainy, Sunny: 0.6*(0.1)*0.7*(0.1)*0.3*(0.6) = 0.000756
Rainy, Sunny, Rainy: 0.6*(0.1)*0.3*(0.6)*0.4*(0.1) = 0.000432

Rainy, Rainy, Rainy: 0.6*(0.1)*0.7*(0.1)*0.7*(0.1) = 0.000294

最大機率為: Sunny, Sunny, Sunny
```

### 根據上述條件, 寫出Viterbi應用程式

In [3]:
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry
states = ('sunny', 'rainy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

In [4]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    # 用來儲存計算的機率(V)與所選的路徑(path)   
    V = [{}]
    path = {}
    # Initialize point (t==0)a
    # Only two scenarios for initialization
    # sunny: P(sunny)*P(dry|sunny)
    # rainy: P(rainy)*P(dry|rainy)

    for y in states:
        # 初始情況:
        # 1. 開始為 sunny，觀察情況 dry → 0.4*0.6
        # 2. 開始為 rainy，觀察情況 dry → 0.6*0.1        
        V[0][y] = start_p[y]*emit_p[y][obs[0]]
        path[y] = [y]     
        
    # Run Viterbi for t > 0
    for t in range(1,len(obs)):
        V.append({})
        newpath = {}
        # cur_state is either sunny or rainy
        for cur_state in states:
            (most_prob, most_pre) = max([
                                        (V[t-1][pre_state] * trans_p[pre_state][cur_state] * emit_p[pre_state][obs[t]], 
                                          pre_state)
                                         for pre_state in states
                                        ])
            # [前一天情況的機率]*[在前一天情況為前提下，當天情況的機率]*[在當下情況的觀察狀態機率]
            V[t][cur_state] = most_prob
            newpath[cur_state] = path[most_pre] + [cur_state]
                
        
        # Don't need to remember the old paths
        path = newpath

    (prob, state) = max([(V[len(obs) - 1][final_state], final_state) for final_state in states])
    return (prob, path[state])

In [5]:
result = viterbi(observations,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probatility)

In [6]:
result

(0.031103999999999993, ['sunny', 'sunny', 'sunny'])